<style>
	#LatamBrain {
		font-size: 50px;
		font-weight: bold;
		color: #5390D9;
		position: relative;
		top: 50px;
		opacity: 0;
		animation: fade-in 2s forwards;
	}
	@keyframes fade-in {
		from {
			top: -50px;
			opacity: 0;
		}
		to {
			top: 50px;
			opacity: 1;
		}
	}
</style>

<div style="text-align:center;">
	<h1 id="LatamBrain">LatamBrain</h1>
</div> 

## __TOOL 1 - Recruiter Analysis__:  Analisis de datos

##### _Modelo en el cual el recruiter al buscar un perfil, por ejemplo: Data Engineer  y que teniendo la información de un CV o perfil de postulante con sus “SKILLS” se ingresen y el modelo nos diga si es apto o no, y el porcentaje que se ajusta el postulante a dicho puesto (Data Engineer)_

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

### __Conexion con la base de datos mysql en AWS RDS__

In [4]:
# Crear la conexión
cnx = mysql.connector.connect(
    host='db-latambrain-luis-mysql.ckwcpsorcjut.us-east-2.rds.amazonaws.com',
    user='admin',
    password='zGqS4YymBH9jsMOEFxoG',
    database='sys'
)

In [5]:
# Ejecutar una consulta SQL y guardar los resultados en un DataFrame de Pandas
query = "SELECT * FROM Job"
df = pd.read_sql(query, cnx) #Job table in Sys DataBase

C:\Users\matia\AppData\Local\Temp\ipykernel_31200\890545583.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


In [7]:
cnx.close()

In [6]:
df

,job_title,level,job_type,salary_avg,benefits,onsite_remote,dim_JobCategory,dim_Company,dim_Location,dim_Skills
0,remote ai/ml engineer in wildlife conservation,mid-level,contract,27.5,"career development, health care",remote,ml,the wild neighbors database project,"remote, us","apis, classification, data management, github,..."
1,ai/ml-powered trading strategy researcher (and...,senior-level,full time,150.0,n/a,remote,ml,white wind research,remote,"finance, machine learning, python, research, t..."
2,principal data engineer,senior-level,full time,200.0,equity,remote,engineer,murmuration,"remote, us","agile, architecture, athena, aws, big data, ci..."
3,mlops engineer,mid-level,full time,129.0,"career development, competitive pay, flex hour...",remote,ml,hala systems,"remote, eu and us","architecture, ci/cd, computer science, compute..."
4,ai developer,senior-level,full time,84.0,"career development, flex hours, startup enviro...",onsite,data scientist,lemon.io,ukraine,"caffe, computer science, computer vision, deep..."
...,...,...,...,...,...,...,...,...,...,...
90,aws data scientist engineer,senior-level,full time,157.0,n/a,onsite,engineer,devoteam,"palau-solità i plegamans, spain","airflow, apis, aws, big data, ci/cd, databrick..."
91,senior data engineer,senior-level,full time,147.5,"career development, flex vacation, startup env...",remote,engineer,testlio,remote in emea,"airflow, architecture, aws, business intellige..."
92,data integration developer,senior-level,full time,98.5,"competitive pay, flex hours, flex vacation, te...",onsite,data scientist,pandata group,"madison, united states","aws, azure, computer science, consulting, data..."
93,business data analyst,mid-level,full time,49.5,"flex hours, gear, parental leave, salary bonus...",onsite,analyst,treatwell,"london, england, united kingdom","data quality, excel, kanban, looker, numpy, pa..."


In [8]:
df_model1 = df.drop(['level', 'job_type', 'salary_avg', 'benefits', 'onsite_remote', 'dim_Company', 'dim_Location'], axis=1)

In [9]:
df_model1

,job_title,dim_JobCategory,dim_Skills
0,remote ai/ml engineer in wildlife conservation,ml,"apis, classification, data management, github,..."
1,ai/ml-powered trading strategy researcher (and...,ml,"finance, machine learning, python, research, t..."
2,principal data engineer,engineer,"agile, architecture, athena, aws, big data, ci..."
3,mlops engineer,ml,"architecture, ci/cd, computer science, compute..."
4,ai developer,data scientist,"caffe, computer science, computer vision, deep..."
...,...,...,...
90,aws data scientist engineer,engineer,"airflow, apis, aws, big data, ci/cd, databrick..."
91,senior data engineer,engineer,"airflow, architecture, aws, business intellige..."
92,data integration developer,data scientist,"aws, azure, computer science, consulting, data..."
93,business data analyst,analyst,"data quality, excel, kanban, looker, numpy, pa..."


### _Creacion de los filtros para investigacion del minimo promedio de aptitudes necesarias y las mas utilizadas para crear la funcion con lambda y crear la nueva columna aptitude_

In [43]:
df['dim_JobCategory'].unique()

array(['ml', 'engineer', 'data scientist', 'analyst'], dtype=object)

#### INVESTIGACION PROMEDIO DE APTITUDES ML

In [41]:
# ML
filtro = df_model1['dim_JobCategory'].str.contains('ml')
df_filtrado = df_model1.loc[filtro]

df_filtrado

,job_title,dim_JobCategory,dim_Skills
0,remote ai/ml engineer in wildlife conservation,ml,"apis, classification, data management, github,..."
1,ai/ml-powered trading strategy researcher (and...,ml,"finance, machine learning, python, research, t..."
3,mlops engineer,ml,"architecture, ci/cd, computer science, compute..."
30,senior product manager (ml/ai),ml,"engineering, llms, machine learning, ux"
44,"senior product manager, ml products",ml,"apis, classification, data analysis, engineeri..."


In [52]:
len(df_filtrado)

5

In [61]:
# Contar la cantidad de splits por coma de cada fila en la columna "dim_Skills"        MACHINE LEARNING
num_splits_ML = df_filtrado['dim_Skills'].str.split(',').apply(len)

# Calcular el promedio total de splits por fila
promedio_splits_ML = num_splits_ML.mean()

print("El promedio total de splits por fila es:", promedio_splits_ML)

El promedio total de splits por fila es: 6.6


#### INVESTIGACION PROMEDIO DE APTITUDES DATA ENGINEER

In [50]:
# engineer
filtro1 = df_model1['dim_JobCategory'].str.contains('engineer')
df_filtrado1 = df_model1.loc[filtro1]

df_filtrado1


,job_title,dim_JobCategory,dim_Skills
2,principal data engineer,engineer,"agile, architecture, athena, aws, big data, ci..."
5,machine learning engineer,engineer,"aws, azure, blockchain, crypto, deep learning,..."
11,sr etl/ data engineer,engineer,"agile, architecture, azure, data analysis, dat..."
13,"data engineering co-op, empress therapeutics",engineer,"architecture, aws, biology, ci/cd, computer sc..."
14,lead data engineer (f/h),engineer,"agile, airflow, apis, azure, big data, elt, et..."
16,senior data engineer,engineer,"a/b testing, airflow, data quality, data wareh..."
17,data engineer,engineer,"apis, architecture, big data, data pipelines, ..."
25,software qa engineer - machine learning qe,engineer,"agile, engineering, machine learning, testing"
32,senior data engineer - industry team - m/f/d,engineer,"apis, architecture, big data, bigquery, cassan..."
34,sr staff data engineer,engineer,"architecture, aws, big data, data analysis, da..."


In [51]:
len(df_filtrado1)

41

In [60]:
# Contar la cantidad de splits por coma de cada fila en la columna "dim_Skills"         DATA ENGINEER
num_splits_DE = df_filtrado1['dim_Skills'].str.split(',').apply(len)

# Calcular el promedio total de splits por fila
promedio_splits_DE = num_splits_DE.mean()

print("El promedio total de splits por fila es:", promedio_splits_DE)

El promedio total de splits por fila es: 7.658536585365853


#### INVESTIGACION PROMEDIO DE APTITUDES DATA SCIENTIST

In [44]:
# data scientist
filtro2 = df_model1['dim_JobCategory'].str.contains('data scientist')
df_filtrado2 = df_model1.loc[filtro2]

df_filtrado2

,job_title,dim_JobCategory,dim_Skills
4,ai developer,data scientist,"caffe, computer science, computer vision, deep..."
6,"applied scientist, amazon shoppable videos",data scientist,"a/b testing, big data, computer science, compu..."
8,senior data scientist,data scientist,"a/b testing, airflow, aws, bayesian, computer ..."
9,"director, machine learning, new initiatives",data scientist,"computer science, computer vision, deep learni..."
10,clinical data operator,data scientist,"genetics, privacy, security, testing"
12,senior manager - data analytics,data scientist,"agile, consulting, data analytics"
15,junior data operations specialist (m/f/d),data scientist,"dataops, finance, flink"
18,premaster programm - data science/big data (w/...,data scientist,"big data, data analytics, knime, machine learn..."
21,senior big data consultant,data scientist,"airflow, architecture, athena, aws, big data, ..."
23,machine learning research scientist,data scientist,"computer science, deep learning, engineering, ..."


In [53]:
len(df_filtrado2)

27

In [59]:
# Contar la cantidad de splits por coma de cada fila en la columna "dim_Skills"   DATA SCIENTIST
num_splits_DS = df_filtrado2['dim_Skills'].str.split(',').apply(len)

# Calcular el promedio total de splits por fila
promedio_splits_DS = num_splits_DS.mean()

print("El promedio total de splits por fila es:", promedio_splits_DS)

El promedio total de splits por fila es: 7.0


#### INVESTIGACION PROMEDIO DE APTITUDES DATA ANALYST

In [54]:
# analyst
filtro3 = df_model1['dim_JobCategory'].str.contains('analyst')
df_filtrado3 = df_model1.loc[filtro3]

df_filtrado3

,job_title,dim_JobCategory,dim_Skills
7,business data analyst lead,analyst,"agile, architecture, ci/cd, computer science, ..."
19,data quality analyst,analyst,data quality
20,portuguese language data analyst - canada,analyst,"kpis, machine learning, react"
22,english language data analyst - canada,analyst,"machine learning, react"
26,senior insight analyst,analyst,"business intelligence, economics, excel, genet..."
31,business data analyst,analyst,"consulting, consulting firm, data mining, etl,..."
37,data analyst,analyst,"bigquery, business intelligence, data analysis..."
40,data analyst (cyprus),analyst,"agile, airflow, crypto, data analysis, data qu..."
41,data analyst marketing (stage juillet 2023) - h/f,analyst,"business intelligence, e-commerce, excel, sql"
48,senior data analyst,analyst,"business analytics, business intelligence, clu..."


In [55]:
len(df_filtrado3)

22

In [58]:
# Contar la cantidad de splits por coma de cada fila en la columna "dim_Skills"  ANALYST
num_splits_a = df_filtrado3['dim_Skills'].str.split(',').apply(len)

# Calcular el promedio total de splits por fila
promedio_splits_a = num_splits_a.mean()

print("El promedio total de splits por fila es:", promedio_splits_a)

El promedio total de splits por fila es: 5.954545454545454


## __Ahora procedemos a investigar los unique() de la columna dim_skills con filtro de dim_job_category__

## ML

In [69]:
# Filtro por la palabra "ml" en la columna "dim_JobCategory"
filtro_ml = df_model1['dim_JobCategory'].str.contains('ml')
unique_ml = df_model1.loc[filtro_ml]['dim_Skills'].unique()
print("Aptitudes de ML disponibles en el dataset:\n")
print(unique_ml)


Aptitudes de ML disponibles en el dataset:

['apis, classification, data management, github, keras, machine learning, ml models, +6'
 'finance, machine learning, python, research, trading strategies'
 'architecture, ci/cd, computer science, computer vision, data pipelines, deep learning, devops, +15'
 'engineering, llms, machine learning, ux'
 'apis, classification, data analysis, engineering, json, machine learning, python, +2']


## Engineer

In [90]:

# Filtro por la palabra "engineer" en la columna "dim_JobCategory"
filtro_engineer = df_model1['dim_JobCategory'].str.contains('engineer')
unique_engineer = df_model1.loc[filtro_engineer]['dim_Skills'].unique()
print("Aptitudes de engineer disponibles en el dataset:\n")
print(unique_engineer)

Aptitudes de engineer disponibles en el dataset:

['agile, architecture, athena, aws, big data, ci/cd, data governance, +24'
 'aws, azure, blockchain, crypto, deep learning, docker, gcp, +13'
 'agile, architecture, azure, data analysis, databricks, data quality, db2, +12'
 'architecture, aws, biology, ci/cd, computer science, data analysis, devops, +14'
 'agile, airflow, apis, azure, big data, elt, etl, +7'
 'a/b testing, airflow, data quality, data warehouse, engineering, genetics, machine learning, +6'
 'apis, architecture, big data, data pipelines, engineering, pipelines, python'
 'agile, engineering, machine learning, testing'
 'apis, architecture, big data, bigquery, cassandra, data pipelines, devops, +11'
 'architecture, aws, big data, data analysis, data pipelines, docker, engineering, +12'
 'big data, data governance, data pipelines, data quality, data warehousing, engineering, etl, +10'
 'architecture, avro, aws, computer science, data governance, data pipelines, data warehous

## Data scientist

In [91]:
# Filtro por la palabra "data scientist" en la columna "dim_JobCategory"
filtro_ds = df_model1['dim_JobCategory'].str.contains('data scientist')
unique_ds = df_model1.loc[filtro_ds]['dim_Skills'].unique()
print("Aptitudes de engineer disponibles en el dataset:\n")
print(unique_ds)

Aptitudes de engineer disponibles en el dataset:

['caffe, computer science, computer vision, deep learning, keras, machine learning, ml models, +7'
 'a/b testing, big data, computer science, computer vision, data analysis, data mining, economics, +11'
 'a/b testing, airflow, aws, bayesian, computer vision, deep learning, docker, +14'
 'computer science, computer vision, deep learning, engineering, llms, machine learning, phd'
 'genetics, privacy, security, testing'
 'agile, consulting, data analytics' 'dataops, finance, flink'
 'big data, data analytics, knime, machine learning, power bi, python, tableau, +1'
 'airflow, architecture, athena, aws, big data, consulting, data analytics, +17'
 'computer science, deep learning, engineering, machine learning, mathematics, phd, postdoc, +7'
 'agile, banking, data analytics, finance, machine learning, mysql, nosql, +8'
 'testing' 'airflow, apis, aws, big data, ci/cd, databricks, docker, +12'
 'data visualization, excel, market research, resea

## Analyst

In [92]:

# Filtro por la palabra "analyst" en la columna "dim_JobCategory"
filtro_analyst = df_model1['dim_JobCategory'].str.contains('analyst')
unique_analyst = df_model1.loc[filtro_analyst]['dim_Skills'].unique()
print("Aptitudes de engineer disponibles en el dataset:\n")
print(unique_analyst)

Aptitudes de engineer disponibles en el dataset:

['agile, architecture, ci/cd, computer science, data analysis, data visualization, engineering, +9'
 'data quality' 'kpis, machine learning, react' 'machine learning, react'
 'business intelligence, economics, excel, genetics, research, statistics'
 'consulting, consulting firm, data mining, etl, excel, industrial, power bi, +3'
 'bigquery, business intelligence, data analysis, economics, engineering, jupyter, mathematics, +7'
 'agile, airflow, crypto, data analysis, data quality, finance, git, +4'
 'business intelligence, e-commerce, excel, sql'
 'business analytics, business intelligence, cluster analysis, data management, data mining, data quality, econometrics, +9'
 'data visualization, economics, engineering, excel, looker, mathematics, power bi, +3'
 'architecture, data warehousing, engineering, finance, looker, machine learning, mathematics, +7'
 'security'
 'data warehouse, engineering, finance, pyspark, python, redshift, statis

### _Lista de las aptitudes y su cantidad de repeticion total en la columna dim_skills_

In [94]:
# Crear una lista con todos los valores de la columna "dim_Skills"
skills_list = list(df_model1['dim_Skills'])

# Crear un diccionario vacío para almacenar los splits y su cantidad de ocurrencias
split_count = {}

# Iterar sobre cada valor en la lista y contar la cantidad de ocurrencias de cada split
for i, skill in enumerate(skills_list):
    # Hacer split por comas y recorrer cada elemento resultante
    for split in skill.split(","):
        # Remover espacios en blanco al inicio y final del elemento
        split = split.strip()
        # Incrementar la cantidad de ocurrencias del split en el diccionario
        if split in split_count:
            split_count[split] += 1
        else:
            split_count[split] = 1

    # Hacer split de la columna "dim_JobCategory" y comparar con el dataset original
    job_category = df_model1.loc[i, 'dim_JobCategory']
    if type(job_category) == str:
        job_category_split = job_category.split(",")
        for category in job_category_split:
            if category.strip() in ['ml', 'engineer', 'data scientist', 'analyst']:
                df_split_count.loc[split, 'job_category'] = category.strip()

# Convertir el diccionario en un DataFrame de pandas
df_split_count = pd.DataFrame.from_dict(split_count, orient='index', columns=['aptitude_repetition'])

# Ordenar el DataFrame por la cantidad de ocurrencias de mayor a menor
df_split_count = df_split_count.sort_values(by=['aptitude_repetition'], ascending=False)

# Mostrar todas las filas
pd.set_option('display.max_rows', len(df_split_count))

# Resetear el índice y renombrar la columna "index"
df_split_count = df_split_count.reset_index().rename(columns={'index': 'aptitude'})

# Mostrar el resultado final
df_split_count

,aptitude,aptitude_repetition
0,engineering,37
1,machine learning,27
2,computer science,26
3,big data,23
4,aws,22
5,agile,19
6,airflow,18
7,azure,17
8,architecture,17
9,data analysis,15


# __MODELO FINAL PARA CONSEGUIR EL DATAFRAME USADO EN EL MODELO__

In [ ]:
#Cargar el nuevo dataset


In [ ]:
#Modificacion al dataframe

In [ ]:
#Modelo